# Secion3 実装演習コード

TensorFlowチュートリアル掲載コードを参考に、float16量子化モデルを作成する。量子化なしモデルとの精度、処理速度、サイズ比較を行い、量子化効果を確認する。

参考URL
https://www.tensorflow.org/lite/performance/post_training_float16_quant?hl=ja

In [15]:
# 参考: TensorFlowチュートリアル
#  https://www.tensorflow.org/lite/performance/post_training_float16_quant?hl=ja

import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pathlib

# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_imgs, test_lbls) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_imgs = test_imgs / 255.0

# ----------------
#  通常モデル作成
# ----------------
# Define the model architecture
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(
  train_images,
  train_labels,
  epochs=1,
  validation_data=(test_imgs, test_lbls)
)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# ファイルダンプ
tflite_models_dir = pathlib.Path("./fig_section3/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)
tflite_model_file = tflite_models_dir/"mnist_model.tflite"
tflite_model_file.write_bytes(tflite_model)

# モデルをインタープリタに読み込む
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

# -------------------
#  量子化モデルに変換
# -------------------

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_fp16_model = converter.convert()

# ファイルダンプ
tflite_model_fp16_file = tflite_models_dir/"mnist_model_quant_f16.tflite"
tflite_model_fp16_file.write_bytes(tflite_fp16_model)

# モデルをインタープリタに読み込む
interpreter_fp16 = tf.lite.Interpreter(model_path=str(tflite_model_fp16_file))
interpreter_fp16.allocate_tensors()

# ----------------
#  モデル比較
# ----------------

import time

# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(model_interp, test_images, test_labels):
    input_index = model_interp.get_input_details()[0]["index"]
    output_index = model_interp.get_output_details()[0]["index"]

    # Run predictions on every image in the "test" dataset.
    start = time.time()

    prediction_digits = []
    for test_image in test_images:
        # Pre-processing: add batch dimension and convert to float32 to match with
        # the model's input data format.
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
        model_interp.set_tensor(input_index, test_image)

        # Run inference.
        model_interp.invoke()

        # Post-processing: remove batch dimension and find the digit with highest
        # probability.
        output = model_interp.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)

    elapsed_time = (time.time() - start) * 1000

    # Compare prediction results with ground truth labels to calculate accuracy.
    accurate_count = 0
    for index in range(len(prediction_digits)):
        if prediction_digits[index] == test_labels[index]:
            accurate_count += 1
    accuracy = accurate_count * 1.0 / len(prediction_digits)

    return accuracy, elapsed_time

# 通常モデルの評価
acc_normal, time_normal = evaluate_model(interpreter, test_imgs, test_lbls)

# 量子化モデルの評価
acc_quant, time_quant = evaluate_model(interpreter_fp16, test_imgs, test_lbls)

print("Normal Model : accuracy=", acc_normal, 
        ", time[ms]=", time_normal)

print("Quantization Model(fp16) : accuracy=", acc_quant, 
        ", time[ms]=", time_quant)


1875/1875 [==============================] - 5s 3ms/step - loss: 0.2885 - accuracy: 0.9191 - val_loss: 0.1407 - val_accuracy: 0.9601
INFO:tensorflow:Assets written to: C:\Users\shima\AppData\Local\Temp\tmpg713m5jp\assets


INFO:tensorflow:Assets written to: C:\Users\shima\AppData\Local\Temp\tmpg713m5jp\assets


INFO:tensorflow:Assets written to: C:\Users\shima\AppData\Local\Temp\tmpmciabwus\assets


INFO:tensorflow:Assets written to: C:\Users\shima\AppData\Local\Temp\tmpmciabwus\assets


Normal Model : accuracy= 0.9601 , time[ms]= 1941.8909549713135
Quantization Model(fp16) : accuracy= 0.9601 , time[ms]= 1938.124179840088
